https://dash.plotly.com/

In [1]:
# !pip install dash
# !pip install dash_bootstrap_components
# !pip install_mantine_components
# !pip install plotly

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly
import yfinance as yfi
from dash import Dash, html, callback, Output, Input, dcc
import dash as dash
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc

import mpld3

In [3]:
# !pip install dash
# !pip install dash_bootstrap_components
# !pip install_mantine_components
# !pip install plotly

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly
import yfinance as yfi
from dash import Dash, html, callback, Output, Input, dcc
import dash as dash
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc

import mpld3

In [12]:
BS = "https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css"
app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])


### -------------------------------------------------------- ###
### -----------------EXTRA STUFF - LUKE -------------------- ###
### -------------------------------------------------------- ###
stock_list = ['AAPL','AMZN','NVDA','MSFT','IBM','INTC']

slider_date_yfi_dict = {0:'1d' ,1:'1wk',2:'1mo',
                    3:'6mo',4:'1y',5:'5y'}
slider_date_yfi_interval_dict = {0:['5m','1 Day'],1:['30m','1 Week'],
                                 2:['30m','1 Month'],3:['1h','6 Months'],
                                 4:['1d','1 Year'],5:['1d','5 Years']}

radio_yfi_list = ['Open Price','Volume of Shares Sold','Gross Profit'] ### THIS IS MY BUTTON NAMES
radio_yfi_data_dict = {'Open Price':'Open', 'Volume of Shares Sold':'Volume'}

### -------------------------------------------------------- ###
### -----------------EXTRA STUFF - DURGESH ----------------- ###
### -------------------------------------------------------- ###

radio_yfi_list_tab2 = ['ElasticNet','Volume of Shares Sold','Gross Profit'] ### THIS IS MY BUTTON NAMES


### -------------------------------------------------------- ###
### -----------------EXTRA STUFF - MICHAEL------------------ ###
### -------------------------------------------------------- ###



app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label="Tab 1 - Stock Data Visualizations", children = [
            html.Div([
            html.H1(children='Stock data visualization', style={'textAlign':'center'}),
            dcc.Dropdown(stock_list, 'AMZN', id='DROPDOWN-SELECTION', clearable=False)]),
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='GRAPH-CONTENT',responsive=True))),  
                dbc.Col(html.Div(dcc.RadioItems(options = radio_yfi_list, value = 'Open Price',id = 'RADIO-SELECTION',
                                        style={'textAlign':'center', 'font-size':40}),
                                style={'border':'2px white solid', 'height':'100%'}), width=3)       
                    ]),
            dcc.Slider(0,5,step=None,marks=slider_date_yfi_dict, value=0,id='SLIDER-SELECTION')
                    ]),

                   # ---------------------------------------------- #
                   ### Durgesh code here for structuring the page!!!
                   # ---------------------------------------------- #
        dcc.Tab(label='Tab 2 - Stock price forecasting',
               children = [
                html.Div([
                html.H1(children='Stock Data Forecasting', style={'textAlign':'center'}),
                dcc.Dropdown(stock_list, 'AMZN', id='DROPDOWN-SELECTION-2', clearable=False)]),
                dbc.Row([
                    dbc.Col(html.Div(dcc.Graph(id='GRAPH-CONTENT-2',responsive=True))),  
                    dbc.Col(html.Div(dcc.RadioItems(options = radio_yfi_list_tab2, value = 'Open Price',id = 'RADIO-SELECTION-2',
                                        style={'textAlign':'center', 'font-size':40}),
                                style={'border':'2px white solid', 'height':'100%'}), width=3)]) 
               ]),

                    # --------------------------------------------- #
                   ### Michael code here for structuring the page!!!
                   # ---------------------------------------------- #
        dcc.Tab(label='Tab 3 - Stock price decomposition', children=[
            html.Div([
                html.H1(children='Stock Data Decomposition', style={'textAlign':'center'}),
                dcc.Dropdown(stock_list, 'AMZN', id='DROPDOWN-SELECTION-3', clearable=False)]),
                dbc.Row([
                    dbc.Col(html.Div(dcc.Graph(id='GRAPH-CONTENT-3',responsive=True))),  
                    dbc.Col(html.Div(dcc.RadioItems(options = radio_yfi_list, value = 'Open Price',id = 'RADIO-SELECTION-3',
                                        style={'textAlign':'center', 'font-size':40}),
                                style={'border':'2px white solid', 'height':'100%'}), width=3)]),
            dcc.Slider(0,5,step=None,marks=slider_date_yfi_dict, value=0,id='SLIDER-SELECTION-3')
                    ]),
               ])
    ])



### Function to update the graph on tab 1, which visualizes different aspects of the stocks
@callback(
    Output('GRAPH-CONTENT', 'figure'),
    Input('DROPDOWN-SELECTION', 'value'),
    Input('SLIDER-SELECTION', 'value'),
    Input('RADIO-SELECTION','value')
)

def update_graph(dropdown_selection, slider_selection, radio_selection):
    plt.close()
    if dropdown_selection in stock_list:
        if radio_selection in ['Open Price','Volume of Shares Sold']:
            radio_selection_conv = radio_yfi_data_dict[radio_selection]
            dff = yfi.Ticker(dropdown_selection).history(period = slider_date_yfi_dict[slider_selection],
                                                interval=slider_date_yfi_interval_dict[slider_selection][0])

            ### Need to change the dates to strings to have the plotting not include days don't have data for and it creates
            ### a long gap. THIS COULD BE SPED UP BY SOME OTHER METHOD
            i=0
            date_str_list = []
            for a in dff.index:
                date_str_list.append(str(dff.index[i]).split(' ')[0] + '\n' + str(dff.index[i]).split(' ')[1])
                i+=1
                
            dff['date_str'] = date_str_list
            dff = dff.reset_index()
            dff = dff.set_index('date_str')
            
            fig, ax = plt.subplots(1,1)
            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.plot(dff.index, dff['Open'])
            
            plt.title(dropdown_selection + ' - ' + slider_date_yfi_interval_dict[slider_selection][1], 
                     fontsize=20)
            plt.ylabel(radio_selection, fontsize=16)
            
            plt.xticks(fontsize=8, rotation=45)
            plt.yticks(fontsize=14)
            plt.grid()
            html_matplot = plotly.tools.mpl_to_plotly(fig)
            return html_matplot
        if radio_selection == 'Gross Profit':
            dff = yfi.Ticker(dropdown_selection).financials.loc['Gross Profit']
            
            fig, ax = plt.subplots(1,1)
            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.plot(dff)
            
            plt.title(dropdown_selection + ' - ' + 'Gross Profit ($)', 
                     fontsize=20)
            plt.ylabel('Gross Profit ($)', fontsize=16)
            plt.xticks(fontsize=8, rotation=45)
            plt.yticks(fontsize=14)
            plt.grid()
            html_matplot = plotly.tools.mpl_to_plotly(fig)
            return html_matplot


# ---------------------------------------------- #
### Durgesh code here for running functions!!! ###
# ---------------------------------------------- #

### THIS IS COMMENTED BECAUSE IT NEEDS INPUTS TO RUN PROPERLY. CODE IT UP A BIT AND THEN UNCOMMENT
# ### Function to update the graph on tab 1, which visualizes different aspects of the stocks
# @callback(
#     Output('GRAPH-CONTENT-2', 'figure'),
#     Input('DROPDOWN-SELECTION-2', 'value'),
#     Input('SLIDER-SELECTION-2', 'value'),
#     Input('RADIO-SELECTION-2','value')
# )

# def update_graph_2(dropdown_selection, slider_selection, radio_selection):
#     plt.close()

#     ### Code for training the model goes here!!!!
#     ### Output will be pred_df with an index x days ahead of dff.index, where x is the input to the text box
    
#     radio_selection_conv = radio_yfi_data_dict[radio_selection]
#     dff = yfi.Ticker(dropdown_selection).history(period = slider_date_yfi_dict[slider_selection],
#                                         interval=slider_date_yfi_interval_dict[slider_selection][0])
#     fig, ax = plt.subplots(1,1)

#     plt.plot(pred_df.index, dff['pred'])
#     plt.plot(dff.index, dff['Open'])
    
#     plt.title(dropdown_selection + ' - ' + slider_date_yfi_interval_dict[slider_selection][1], 
#              fontsize=20)
#     plt.ylabel(radio_selection, fontsize=16)
    
#     plt.xticks(fontsize=8, rotation=45)
#     plt.yticks(fontsize=14)
#     plt.grid()
#     html_matplot = plotly.tools.mpl_to_plotly(fig)
#     return html_matplot


# ---------------------------------------------- #
### Michael code here for running functions!!! ###
### COPY MY GRAPH AND INSTEAD OF PLOT MY DATA PLOT THE DECOMPOSITION ###
# ---------------------------------------------- #


### Function to update the graph on tab 3, which visualizes different aspects of the stock decomposition
@callback(
    Output('GRAPH-CONTENT-3', 'figure'),
    Input('DROPDOWN-SELECTION-3', 'value'),
    Input('SLIDER-SELECTION-3', 'value'),
    Input('RADIO-SELECTION-3','value')
)

def update_graph_3(dropdown_selection3, slider_selection3, radio_selection3):
    plt.close()
    if dropdown_selection3 in stock_list:
        if radio_selection3 in ['Open Price','Volume of Shares Sold']:
            radio_selection_conv = radio_yfi_data_dict[radio_selection3]
            dff = yfi.Ticker(dropdown_selection3).history(period = slider_date_yfi_dict[slider_selection3],
                                                interval=slider_date_yfi_interval_dict[slider_selection3][0])
            
            ### Need to change the dates to strings to have the plotting not include days don't have data for and it creates
            ### a long gap. THIS COULD BE SPED UP BY SOME OTHER METHOD
            i=0
            date_str_list = []
            for a in dff.index:
                date_str_list.append(str(dff.index[i]).split(' ')[0] + '\n' + str(dff.index[i]).split(' ')[1])
                i+=1
                
            dff['date_str'] = date_str_list
            dff = dff.reset_index()
            dff = dff.set_index('date_str')
            ### CHANGE BELOW TO PLOT THE DECOMPOSITION!!!
            fig, ax = plt.subplots(1,1)
            plt.plot(dff.index, dff['Open'])
            
            plt.title(dropdown_selection3 + ' - ' + slider_date_yfi_interval_dict[slider_selection3][1], 
                     fontsize=20)
            plt.ylabel(radio_selection3, fontsize=16)
            
            plt.xticks(fontsize=8, rotation=45)
            plt.yticks(fontsize=14)
            plt.grid()
            html_matplot = plotly.tools.mpl_to_plotly(fig)
            return html_matplot
        if radio_selection3 == 'Gross Profit':
            dff = yfi.Ticker(dropdown_selection3).financials.loc['Gross Profit']
            
            fig, ax = plt.subplots(1,1)                        
            ### CHANGE BELOW TO PLOT THE DECOMPOSITION!!!
            plt.plot(dff)
            
            plt.title(dropdown_selection3 + ' - ' + 'Gross Profit ($)', 
                     fontsize=20)
            plt.ylabel('Gross Profit ($)', fontsize=16)
            plt.xticks(fontsize=8, rotation=45)
            plt.yticks(fontsize=14)
            plt.grid()
            html_matplot = plotly.tools.mpl_to_plotly(fig)
            return html_matplot


if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='tab')



Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

C:\Users\lvden\anaconda3\envs\Data-analysis\Lib\site-packages\plotly\matplotlylib\renderer.py:645: UserWarning:

Looks like the annotation(s) you are trying 
to draw lies/lay outside the given figure size.

Therefore, the resulting Plotly figure may not be 
large enough to view the full text. To adjust 
the size of the figure, use the 'width' and 
'height' keys in the Layout object. Alternatively,
use the Margin object to adjust the figure's margins.

C:\Users\lvden\anaconda3\envs\Data-analysis\Lib\site-packages\plotly\matplotlylib\renderer.py:645: UserWarning:

Looks like the annotation(s) you are trying 
to draw lies/lay outside the given figure size.

Therefore, the resulting Plotly figure may not be 
large enough to view the full text. To adjust 
the size of the figure, use the 'width' and 
'height' keys in the Layout object. Alternatively,
use the Margin object to adjust the figure's margins.

C:\Users\lvden\anaconda3\envs\Data-analysis\Lib\site-packages\plotly\matplotlylib\render

# To do
- ElasticNet code simplified for Durgesh

In [ ]:
### ElasticNet code
df_part1 = yfi.Ticker(STOCK SYMBOL).history(period = '1Y', interval='1d')
df_part2 = yfi.Ticker(STOCK SYMBOL).financials.loc['Gross Profit']

### Merge df_part1 and df_part2 where left is df_part1 and do on left and ffill the na in the merged df. 
### If there are still na at top then bfill
X = df_part1.drop(['Open'], axis=1)
y = df_part1['Open']
model = ElasticNet(alpha=alpha, l1_ratio = l1_ratio)
model.fit(X=X, y=y)
### Need to look at forecast, not predict


### Then graph with same graph code I have and add the 
